In [ ]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import networkx as nx
from pathlib import Path
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', 500)

from sklearn.preprocessing import MinMaxScaler
import sys
sys.path.append('/Users/david/Dropbox/PhD/Scripts/Spatial analyses')
import pyspace
import libpysal as lps
from scipy.spatial import cKDTree
from libpysal.weights.distance import get_points_array
from esda import fdr
from importlib import reload
pd.set_option('display.max_rows', 500)
reload(pyspace)
import seaborn as sns
from esda.moran import Moran
# sns.set_theme(font = 'Helvetica')
%matplotlib inline
from numba import NumbaDeprecationWarning
import warnings
import os
# Suppress NumbaDeprecationWarning
warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)
from utils import read_data, plot_covariate_distributions, plot_match, compare_balance, sizeof_fmt, optimize_memory_df, plot_categorical_proportional_diff, compute_mean_differences_and_proportions, love_plot, sensitivity_analysis_k_neighbors

In [ ]:
data_folder  = Path('../Data/')
main_folder = Path('../Manuscript/Economic analysis of integrative medicine/')
results_folder = main_folder/'Results'
model_folder = results_folder/'Models'

if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [ ]:
# data = pd.read_parquet('../Data/processed/full_dataset_nonull.parquet.gzip')

In [ ]:
# data.filter(regex='mean_pm10')

In [ ]:
# del data

In [ ]:
df_treated_filtered = pd.read_parquet('../Data/processed/df_treated_filtered_nominors.parquet.gzip')
df_treated_filtered_t_1 = pd.read_parquet('../Data/processed/df_treated_filtered_t_1_nominors.parquet.gzip')

In [ ]:
df_treated_filtered.uuid.nunique()

In [ ]:
df_treated_filtered.groupby(['NOANNEE','CDPHYSSEXE'])['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack(level=1).div(1000).round(1)

In [ ]:
test = pd.concat([df_treated_filtered.groupby(['NOANNEE','CDPHYSSEXE'])['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1),
df_treated_filtered.groupby(['NOANNEE','age_group'],observed=True)['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1),
df_treated_filtered.groupby(['NOANNEE','ssep3_q'],observed=True)['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1),
df_treated_filtered.groupby(['NOANNEE','Language'],observed=True)['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1),
df_treated_filtered.groupby(['NOANNEE','CAREMODEL'],observed=True)['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1),
df_treated_filtered.groupby(['NOANNEE','MTFRANCHISECOUV'],observed=True)['PRESTATIONS_BRUTES_AOS'].agg({"mean","std"}).unstack().div(1000).round(1)], axis=1).T.reset_index()
test.columns = ['Statistic', 'Categories', '2017', '2018', '2019', '2020', '2021']

# Pivot the DataFrame to make each row represent one year and one sex
df_pivot = test.pivot(index='Categories', columns='Statistic')

for year in ['2017', '2018', '2019', '2020', '2021']:
    df_pivot[year] = df_pivot[(year, 'mean')].astype(str) + " (" + df_pivot[(year, 'std')].astype(str) + ")"
df_pivot.columns = df_pivot.columns.droplevel(1)  # Drops the upper level, leaving only the years
df_final = df_pivot.reset_index()
df_final = df_final.loc[:,~df_final.columns.duplicated()].copy()
df_final.to_clipboard()

In [ ]:
df_treated_filtered = gpd.GeoDataFrame(df_treated_filtered, crs = 2056, geometry = gpd.points_from_xy(df_treated_filtered['E'], df_treated_filtered['N']))
df_treated_filtered.sample(10000).plot('ssep3_q', categorical = True, markersize = 3, cmap='bwr', legend=True)

In [ ]:
def percentile_25(x):
    return np.percentile(x, 25)

def percentile_75(x):
    return np.percentile(x, 75)

def median(x):
    return np.median(x)


In [ ]:
test = pd.concat([df_treated_filtered.groupby(['NOANNEE','CDPHYSSEXE'])['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','age_group'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','ssep3_q'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','Language'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','CAREMODEL'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','MTFRANCHISECOUV'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"mean","std"}).unstack().round(1)], axis=1).T.reset_index()
test.columns = ['Statistic', 'Categories', '2017', '2018', '2019', '2020', '2021']

# Pivot the DataFrame to make each row represent one year and one sex
df_pivot = test.pivot(index='Categories', columns='Statistic')

for year in ['2017', '2018', '2019', '2020', '2021']:
    df_pivot[year] = df_pivot[(year, 'mean')].astype(str) + " (" + df_pivot[(year, 'std')].astype(str) + ")"
df_pivot.columns = df_pivot.columns.droplevel(1)  # Drops the upper level, leaving only the years
df_final = df_pivot.reset_index()
df_final = df_final.loc[:,~df_final.columns.duplicated()].copy()
df_final.to_clipboard()

In [ ]:
pd.concat([df_treated_filtered.groupby(['NOANNEE','CDPHYSSEXE'])['PRESTATIONS_BRUTES_LCA'].agg({"median","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','age_group'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"median","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','ssep3_q'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"median","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','CDLANGUE'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"median","std"}).unstack().round(1),
df_treated_filtered.groupby(['NOANNEE','CAREMODEL'],observed=True)['PRESTATIONS_BRUTES_LCA'].agg({"median","std"}).unstack().round(1)], axis=1).T.reset_index()

In [ ]:
# # Format the result as 'p50 [p25, p75]'
# table_costs['p50 [p25, p75]'] = table_costs.apply(lambda row: f"{row['median']} [{row['percentile_25']}, {row['percentile_75']}]", axis=1)

# # Select only the formatted column
# table_costs = table_costs[['p50 [p25, p75]']]


In [ ]:
df_uuid = df_treated_filtered[['uuid']]

df_uuid = df_uuid.drop_duplicates().reset_index(drop=True)

In [ ]:
yearly_data_AOS = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_AOS'].sum().div(1000000).reset_index()
yearly_data_LCA = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_LCA'].sum().div(1000000).reset_index()
yearly_data_CAM = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_CAM'].sum().div(1000000).reset_index()

# Normalize data

yearly_data_AOS['PRESTATIONS_BRUTES_AOS'] = yearly_data_AOS['PRESTATIONS_BRUTES_AOS'] / yearly_data_AOS['PRESTATIONS_BRUTES_AOS'].iloc[0] * 100
yearly_data_LCA['PRESTATIONS_BRUTES_LCA'] = yearly_data_LCA['PRESTATIONS_BRUTES_LCA'] / yearly_data_LCA['PRESTATIONS_BRUTES_LCA'].iloc[0] * 100
yearly_data_CAM['PRESTATIONS_BRUTES_CAM'] = yearly_data_CAM['PRESTATIONS_BRUTES_CAM'] / yearly_data_CAM['PRESTATIONS_BRUTES_CAM'].iloc[0] * 100

# Plot normalized data
fig, ax = plt.subplots(figsize=(8, 4))
sns.lineplot(data=yearly_data_AOS, x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', marker='o', label='CM - MHI', ax=ax)
sns.lineplot(data=yearly_data_LCA, x='NOANNEE', y='PRESTATIONS_BRUTES_LCA', marker='o', label='CAM - SI', ax=ax)
sns.lineplot(data=yearly_data_CAM, x='NOANNEE', y='PRESTATIONS_BRUTES_CAM', marker='o', label='CAM - MHI', ax=ax)
ax.set_xticks(yearly_data_CAM['NOANNEE'])  # Assuming x-values are year numbers
ax.set_xticklabels(yearly_data_CAM['NOANNEE'].astype('string'), rotation=45)
ax.grid(True)

plt.xlabel('Year')
plt.ylabel('Evolution of annual expenditures (%)')  # Update label to indicate these are percentages
plt.grid(True)
plt.legend(title='Prestation type')
# plt.show()
plt.savefig(results_folder/'Figures_rapport/Evolution_costs_perc_baseline.png', dpi = 300, bbox_inches='tight')

In [ ]:
yearly_data_CAM

In [ ]:
yearly_data_LCA

In [ ]:
yearly_data_AOS = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_AOS'].sum().div(1000000).reset_index()
yearly_data_LCA = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_LCA'].sum().div(1000000).reset_index()
yearly_data_CAM = df_treated_filtered.groupby('NOANNEE')['PRESTATIONS_BRUTES_CAM'].sum().div(1000000).reset_index()

# Create a long format DataFrame for easier plotting with seaborn
combined_data = pd.concat([
    yearly_data_AOS.rename(columns={'PRESTATIONS_BRUTES_AOS': 'Values'}).assign(Type='CM - MHI'),
    yearly_data_CAM.rename(columns={'PRESTATIONS_BRUTES_CAM': 'Values'}).assign(Type='CAM - MHI'),
    yearly_data_LCA.rename(columns={'PRESTATIONS_BRUTES_LCA': 'Values'}).assign(Type='CAM - SI')
])

# Adjusted for FacetGrid with custom x-ticks
def plot_facet_grid(data, x, y, facet_col, x_label, y_label):
    g = sns.FacetGrid(data, row=facet_col, sharey=False, height=4, aspect=2)
    g.map_dataframe(sns.lineplot, x=x, y=y, marker='o')
    g.set_axis_labels(x_label, y_label)

    # Set x-ticks for each facet
    for ax in g.axes.flatten():
        ax.set_xticks(data[x].unique())  # Assuming x-values are year numbers
        ax.set_xticklabels(data[x].unique(), rotation=45)
        ax.grid(True)

# Combine and rename data as per previous example for combined_data
plot_facet_grid(
    combined_data,
    'NOANNEE', 'Values', 'Type',
    'Year', 'Total annual expenditures (millions CHF)'
)
plt.savefig(results_folder/'Figures_rapport/Evolution_costs.png', dpi = 300, bbox_inches='tight')

## Treatment defitions

'treatment','n_month_lca_by_patient','n_dispensateur_lca','month_reached_1000_lca'

Would be interesting to check whether Cancer patients tend to have less spending in Pain medication !

In [ ]:
df_treated_filtered[df_treated_filtered.ihs_cost_aos > 0].groupby('uuid').size().value_counts()

### Cost distribution after transformation

#### AOS

In [ ]:
df_treated_filtered['n_atc'].plot.hist(bins = 100)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 4))

g = sns.kdeplot(data=df_treated_filtered, x="PRESTATIONS_BRUTES_AOS", ax = axs[0])
g.set_xlabel('Yearly CM claims amount (MHI) (CHF)')
g = sns.kdeplot(data=df_treated_filtered, x="ihs_cost_aos", ax = axs[1])
g.set_xlabel('IHS transformed yearly CM claims amount (MHI) (CHF)')
g = sns.kdeplot(data=df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_AOS > df_treated_filtered.MTFRANCHISECOUV], x="ihs_cost_aos", ax = axs[2])
g.set_xlabel('IHS transformed yearly CM claims amount (MHI) (CHF)')
plt.savefig(results_folder/'Figures_rapport/IHS_transformation_procedure_AOS.png', dpi = 300, bbox_inches='tight')

In [ ]:
df_treated_filtered['ihs_cost_aos'].plot.kde()

#### LCA

In [ ]:
df_treated_filtered['ihs_cost_lca'].plot.kde()

## Création de sous-groupes

- Répondre à des questions spécifiques
- Augmenter l'homogénéité de la population étudiée

In [ ]:
# Healthy = People with PCG flags = 0 in any year
uuid_healthy = df_treated_filtered[df_treated_filtered.n_flags == 0].uuid.unique()
df_healthy = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_healthy)]

# Healthy = People with AOS = 0 at baseline
uuid_healthy_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.n_flags == 0)].uuid.unique()
df_healthy_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_healthy_bs)]

# Patients with multimorbidity
uuid_multimorbid = df_treated_filtered[df_treated_filtered.multimorbidity == 1].uuid.unique()
df_multimorbid = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_multimorbid)]

# Patients with multimorbidity at baseline
uuid_multimorbid_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.multimorbidity == 1)].uuid.unique()
df_multimorbid_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_multimorbid_bs)]

# Patients with cancer
uuid_cancer = df_treated_filtered[(df_treated_filtered.Cancer_PCG == 1)].uuid.unique()
df_cancer = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_cancer)]

# Patients with cancer at baseline
uuid_cancer_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.Cancer_PCG == 1)].uuid.unique()
df_cancer_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_cancer_bs)]

# Patients with diabetes
uuid_diab = df_treated_filtered[(df_treated_filtered.Diabetes_PCG == 1)].uuid.unique()
df_diab = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_diab)]

# Patients with diabetes at baseline
uuid_diab_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.Diabetes_PCG == 1)].uuid.unique()
df_diab_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_diab_bs)]

# Patients with pain
uuid_pain = df_treated_filtered[(df_treated_filtered.Pain_PCG == 1)].uuid.unique()
df_pain = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_pain)]

# Patients with pain
uuid_mental = df_treated_filtered[(df_treated_filtered.Mental_PCG == 1)].uuid.unique()
df_mental = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_mental)]


In [ ]:
df_diab.groupby('locdrhosp')['n_flags'].mean()

## Total spending by category

In [ ]:
variable_names = pd.DataFrame({"old": ['multimorbidity','ssep3_q','Urbanicity_simple','n_atc','n_flags','NBAGE',"NBAGE_std",'age_group', "ssep3_std",'ssep3', 'region_DE', 'region_FR', 'region_IT','urb_Urbain','urb_Périurbain','Asthma_PCG', 'Cancer_PCG', 'Diabetes_PCG', 'Epilepsy_PCG',
       'Glaucoma_PCG', 'HIV_AIDS_PCG', 'Heart_disease_PCG',
       'Hypertension_related_PCG', 'Immune_PCG', 'Inflammatory_PCG',
       'Mental_PCG', 'Other_PCG', 'Pain_PCG', 'Parkinson_PCG', 'Thyroid_PCG', "SEX_F",'SEX','CDPHYSSEXE','LANG', "cds_std",'cds','LANG_FR','D_MEDIC_B','D_MEDIC_S','D_MEDIC_B_std','D_MEDIC_S_std','DEDUCTIBLE_above_500','E_std','N_std','E_std:N_std','PRESTATIONS_BRUTES_ATC','PRESTATIONS_TOTAL','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','PRESTATIONS_BRUTES_CAM','PRESTATIONS_BRUTES_AMBULATOIRE','PRESTATIONS_BRUTES_STATIONNAIRE','PRESTATIONS_ACCIDENT','PRESTATIONS_DISEASE','PRESTATIONS_BIRTH','MTFRANCHISECOUV','mean_pm10','mean_no2','mean_pm25','mean_ndvi','mean_lst','mean_carnight'],
                           "new": ['Multimorbidity','Swiss-SEP','Urbanicity','Number of ATC','Number of PCG flags','Age',"Age", 'Age Group', "SES index",'SES index','German', 'French', 'Italian','Urban','Periurban', 'Asthma', 'Cancer', 'Diabetes', 'Epilepsy', 'Glaucoma', 'HIV/AIDS',
       'Heart disease', 'Hypertension related', 'Immune', 'Inflammatory',
       'Mental', 'Other', 'Pain', 'Parkinson', 'Thyroid', "Sex (Female)",'Sex','Sex','Langage', "CDS",'CDS','French speaker','Access to prim. care med.','Access to spec. med.','Access to prim. care med.','Access to spec. med.','Deductible (>500)','E','N','E:N','Drug-related claims (CHF)','Total claims amount (CHF)','CM claims (MHI) (CHF)','CAM claims (SI) (CHF)','CAM claims (MHI) (CHF)','Ambulatory claims (CHF)','Stationary claims (CHF)','Accident-related claims (CHF)','Disease-related claims (CHF)','Birth-related claims (CHF)','Deductible','PM10','NO2','PM25','NDVI','LST','Nighttime car noise']})
rename_dict = variable_names.set_index('old')['new'].to_dict()

def update_variable_names(summary_table, variable_names, table_type):
    name_mapper = variable_names.set_index('old')['new'].to_dict()
    if table_type == 'summary':
        name_mapper = {f"{key}, mean (SD)": f"{value}, mean (SD)" for key, value in name_mapper.items()}
    elif table_type == 'categorical':
        name_mapper = {f"{key}, n (%)": f"{value}, n (%)" for key, value in name_mapper.items()}
    summary_table = summary_table.rename(index=name_mapper)
    return summary_table

In [ ]:
df_treated_filtered_long = df_treated_filtered.set_index('NOANNEE')[['PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_CAM','PRESTATIONS_BRUTES_LCA']].stack().reset_index()

df_treated_filtered_long.columns = ['NOANNEE','cat_spending','amount']

# sns.barplot(df_treated_filtered_long, x="NOANNEE", y="amount", hue = 'cat_spending')

In [ ]:
# Assuming df_treated_filtered_long is your DataFrame after manipulation
df_treated_filtered_long['cat_spending'] = df_treated_filtered_long['cat_spending'].replace({
    'PRESTATIONS_BRUTES_AOS': 'Conventional medicine (MHI)',
    'PRESTATIONS_BRUTES_CAM': 'Complementary medicine (MHI)',
    'PRESTATIONS_BRUTES_LCA': 'Complementary medicine (SI)'
})

In [ ]:
palette = {
    'Conventional medicine (MHI)': '#4E79A7',
    'Complementary medicine (MHI)': '#59A14F',
    'Complementary medicine (SI)': '#F28E2B'
}

In [ ]:
# Now plot with Seaborn
g = sns.barplot(data=df_treated_filtered_long, x="NOANNEE", y="amount", hue='cat_spending', palette=palette)
# plt.ylim([0,10000])
# g.set_yscale("log")
# the non-logarithmic labels you want
# ticks = [1, 10, 100, 1000, 10000, 100000]
# g.set_yticks(ticks)
# g.set_yticklabels(ticks)
g.set_xlabel('Year')
plt.legend(title='')
# If you want to further customize the legend, you can use plt.legend:
# plt.legend(title='S')
plt.ylabel('Average yearly claims amount (CHF)')
# To ensure the plot displays properly in Jupyter notebooks
plt.savefig(results_folder/'Figures_rapport/Avg_expenditures.png', dpi = 300, bbox_inches='tight')

In [ ]:
# Now plot with Seaborn
g = sns.barplot(data=df_treated_filtered_long, x="NOANNEE", y="amount", hue='cat_spending', palette=palette)
# plt.ylim([0,10000])
g.set_yscale("log")
# the non-logarithmic labels you want
ticks = [1, 10, 100, 1000, 10000, 100000]
g.set_yticks(ticks)
g.set_yticklabels(ticks)
g.set_xlabel('Year')
plt.legend(title='')
# If you want to further customize the legend, you can use plt.legend:
# plt.legend(title='S')
plt.ylabel('Average yearly claims amount (CHF)')
# To ensure the plot displays properly in Jupyter notebooks
plt.savefig(results_folder/'Figures_rapport/Avg_expenditures_log.png', dpi = 300, bbox_inches='tight')

In [ ]:
# First subplot
g = sns.lineplot(x='NOANNEE', y='amount', hue='cat_spending', data=df_treated_filtered_long, marker='o')
g.set_yscale("log")
# the non-logarithmic labels you want
ticks = [1, 10, 100, 1000, 10000, 100000]
g.set_yticks(ticks)
g.set_yticklabels(ticks)
g.set_xlabel('Year')
plt.legend(title='')
# axs[0, 0].set_title('All')
# axs[0, 0].set_xlabel('Year')
# axs[0, 0].set_ylabel('Average Number of ATCs')
# axs[0, 0].legend(title='Treatment', loc='upper left')
# axs[0, 0].grid(True)

In [ ]:
df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_CAM > 0].groupby('NOANNEE').uuid.nunique().plot.bar()

In [ ]:
# sns.barplot(
#     x=df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_CAM > 0]['NOANNEE'], 
#     y=df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_CAM > 0]['PRESTATIONS_TOTAL'], 
#     hue=df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_CAM > 0]['cat_spending'].map(rename_dict)
# )

In [ ]:
df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_LCA > 0].groupby('NOANNEE').uuid.nunique().plot.bar()

In [ ]:
g = sns.barplot(df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_CAM > 0], x="NOANNEE", y="PRESTATIONS_BRUTES_CAM", color = '#59A14F')
g.set_ylabel('Average yearly claims amount (CHF)')
# g.legend(title='Complementary medicine (LAMal) for users')
g.set_xlabel('Year')
g.set_ylim([0,1500])
plt.savefig(results_folder/'Figures_rapport/Avg_expenditures_CAM_users.png', dpi = 300, bbox_inches='tight')

In [ ]:
df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_CAM > 0].groupby('year')['PRESTATIONS_BRUTES_CAM'].describe()

In [ ]:
sns.barplot(df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_LCA > 0], x="NOANNEE", y="PRESTATIONS_BRUTES_LCA")

### Number of individuals by cumulated years of treatment

In [ ]:
ttmt_month_gp_uuid = df_treated_filtered.groupby('uuid')['n_month_lca_by_patient'].sum().to_dict()
df_treated_filtered['cumulated_n_month_lca_by_patient'] = df_treated_filtered['uuid'].map(ttmt_month_gp_uuid)
df_uuid['cumulated_n_month_lca_by_patient'] = df_uuid['uuid'].map(ttmt_month_gp_uuid)

In [ ]:
df_treated_filtered.groupby('uuid')['PRESTATIONS_BRUTES_LCA'].sum().plot.hist(bins = 100)

In [ ]:
# df_treated_filtered.groupby('treatment_cumulated_lca_cam').uuid.nunique().plot.bar()
from matplotlib.ticker import FuncFormatter

# First, perform your groupby operation and reset the index to turn it into a DataFrame
df_aggregated = df_treated_filtered.groupby('treatment_cumulated_lca_cam')['uuid'].nunique().reset_index()

# Rename the columns for clarity if desired
df_aggregated.columns = ['Treatment', 'UniqueCount']
# Now, use Seaborn to create a bar plot
def thousands_separator(x, pos):
    return '{:,.0f}'.format(x)  # The '0f' ensures no decimal places

# Assuming df_aggregated is your DataFrame after the groupby and reset_index
sns.barplot(x='Treatment', y='UniqueCount', data=df_aggregated, palette = 'Blues')

# Get the current axis
ax = plt.gca()

# Set the formatter for the y-axis to use your custom formatter
ax.yaxis.set_major_formatter(FuncFormatter(thousands_separator))

# Optional: add labels and title for clarity
plt.xlabel('Number of years in the treatment group (CAM claims amount > 0) \n (MHI and SI combined)')
plt.ylabel('Number of individuals')
# plt.title('Number of individuals by Treatment')
plt.savefig(results_folder/'Figures_rapport/Number_users_CAM_cumulated_years.png', dpi = 300, bbox_inches='tight')

In [ ]:
df_aggregated['perc_count'] = (df_aggregated['UniqueCount']/df_aggregated['UniqueCount'].sum())*100

In [ ]:
df_treated_filtered.groupby('cumulated_n_month_lca_by_patient').uuid.nunique().plot.bar(figsize = (12,8))

In [ ]:
df_uuid['cumulated_n_month_lca_by_patient'].median()

## What generates spending in AOS?

#### Time-varying covariates (proxies for health status)
- Age
- Sex
- Deductible
- Education, income (Swiss SES index)
- Access to healthcare
    - Captured directly through measured access (existing variables : d_medic_s, d_medic)
    - Captured indirectly through urban contexts (urban, periurban, rural)
- Disease state : CDS, PCGs, HOSPITALISATIONS

#### Time-invariant covariates
- Sociocultural norms:
    - Captured through the canton of residence and spoken langage
- Policies :
    - Captured through the canton of residence 
- Environmental factors:
    - Air quality (NO2, PM10, PM2.5)
    - Vegetation (NDVI)
    - Noise (dB)

#### Time-varying treatment
- Treatment


#### Temporal lags

- To explore ! But that would result in losing at least the baseline year to be able to create the lagged variables.


### Health status -> cost, treatment

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_AOS", x='n_flags', width=.6, palette="vlag", showfliers=False, ax=axs[0, 0])

# Second subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_DISEASE", x='n_flags', width=.6, palette="vlag", showfliers=False, ax=axs[0, 1])

# Third subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_LCA", x='n_flags', width=.6, palette="vlag", showfliers=False, ax=axs[1, 0])

# Fourth subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_AOS", x='locdrhosp', width=.6, palette="vlag", showfliers=False, ax=axs[1, 1])

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(6, 4))

# First subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_AOS", x='multimorbidity', width=.6, palette="vlag", showfliers=False, ax=axs[0,0])

# Second subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_LCA", x='multimorbidity', width=.6, palette="vlag", showfliers=False, ax=axs[0,1])

# Third subplot
sns.barplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_CAM", x='multimorbidity', width=.6, palette="vlag", ax=axs[1,0])

# Fourth subplot
sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_DISEASE", x='multimorbidity', width=.6, palette="vlag", showfliers=False, ax=axs[1,1])


# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
df_treated_filtered.groupby('multimorbidity')['PRESTATIONS_BRUTES_CAM'].mean()

#### Remarques

- Les personnes avec des comorbidités ne sont pas celles qui utilisent plus de LCA
- Les personnes avec des comorbidités dépensent bcp plus en AOS (logique)

In [ ]:
df_treated_filtered['CDPHYSSEXE']

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(10, 6))

# First subplot
g = sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_AOS", x='CDPHYSSEXE', width=.6, palette="vlag", showfliers=False, ax=axs[0, 0])
g.set_xlabel('Sex')
g.set_ylabel('Annual CM claims amount (MHI) (CHF)')
# Second subplot
g = sns.boxplot(data=df_treated_filtered, y="cds", x='CDPHYSSEXE', width=.6, palette="vlag", showfliers=False, ax=axs[0, 1])
g.set_xlabel('Sex')
g.set_ylabel('Chronic Disease Score (CHF)')
# Third subplot
g = sns.barplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_CAM", x='CDPHYSSEXE', width=.6, palette="vlag", ax=axs[1, 0])
g.set_xlabel('Sex')
g.set_ylabel('Annual CAM claims amount (MHI) (CHF)')
# Fourth subplot
g = sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_LCA", x='CDPHYSSEXE', width=.6, palette="vlag",showfliers=False, ax=axs[1, 1])
g.set_xlabel('Sex')
g.set_ylabel('Annual CAM claims amount (SI) (CHF)')
# Adjust layout
plt.tight_layout()
plt.savefig(results_folder/'Figures_rapport/Fourplot_cost_by_sex.png', dpi = 300, bbox_inches='tight')

### Age -> cost, disease state, treatment

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(10, 6))

# First subplot
g = sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_AOS", x='age_group', width=.6, palette="vlag", showfliers=False, ax=axs[0, 0])
g.set_xlabel('Age group')
g.set_ylabel('Annual CM claims amount (MHI) (CHF)')
# Second subplot
g = sns.boxplot(data=df_treated_filtered, y="cds", x='age_group', width=.6, palette="vlag", showfliers=False, ax=axs[0, 1])
g.set_xlabel('Age group')
g.set_ylabel('Chronic Disease Score (CHF)')
# Third subplot
g = sns.barplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_CAM", x='age_group', width=.6, palette="vlag", ax=axs[1, 0])
g.set_xlabel('Age group')
g.set_ylabel('Annual CAM claims amount (MHI) (CHF)')
# Fourth subplot
g = sns.boxplot(data=df_treated_filtered, y="PRESTATIONS_BRUTES_LCA", x='age_group', width=.6, palette="vlag",showfliers=False, ax=axs[1, 1])
g.set_xlabel('Age group')
g.set_ylabel('Annual CAM claims amount (SI) (CHF)')
# Adjust layout
plt.tight_layout()
plt.savefig(results_folder/'Figures_rapport/Fourplot_cost_by_age.png', dpi = 300, bbox_inches='tight')

### Year & Year-1 -> cost, disease state, treatment

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.kdeplot(data=df_treated_filtered.head(100000), x="treatment", y="treatment_t_1", ax=axs[0, 0])

# Second subplot
sns.kdeplot(data=df_treated_filtered.head(100000), x="ihs_cost_aos", y="ihs_cost_aos_t_1", hue='treatment', ax=axs[0, 1])

# Third subplot
sns.kdeplot(data=df_treated_filtered.head(100000), x="ihs_cost_lca", y="ihs_cost_lca_t_1", ax=axs[1, 0])

# Fourth subplot
sns.kdeplot(data=df_treated_filtered.head(100000), x="ihs_cost_lca", y="ihs_cost_aos", ax=axs[1, 1])

# Adjust layout
plt.tight_layout()
plt.show()

## Time -> Cost, Disease, Treament

In [ ]:
palette = sns.color_palette()


dic_labels = {1:'Usage',0:'No usage'}

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# List of dataframes and titles
dataframes = [df_treated_filtered, df_healthy, df_multimorbid, df_cancer]
titles = ['All individuals', 'Individuals without chronic conditions', 'Multimorbid individuals', 'Individuals with cancer']

# Create subplots
for i, (ax, df, title) in enumerate(zip(axs.flat, dataframes, titles)):
    sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cam_only', data=df, marker='o', ax=ax, legend=False)
    ax.set_title(title)
    ax.set_xlabel('Year')
    ax.set_ylabel('Conventional Medicine\nexpenditures (CHF)')
    ax.grid(True)

# Manually create legend handles and labels
legend_handles = [plt.Line2D([0], [0], marker='o', color=palette[i], 
                             label=dic_labels[treatment], markersize=8, linestyle='-') 
                  for i, treatment in enumerate(df_treated_filtered['treatment_cam_only'].unique())]

# Create a single legend with the correct title
legend = fig.legend(handles=legend_handles, 
           title='Complementary/Alternative Medicine (CAM)\nthrough Mandatory Health Insurance (MHI)', 
           loc='lower center', alignment='center', bbox_to_anchor=(0.5, -0.04), ncol=2)
plt.setp(legend.get_title(), ha='center')  # 'ha' stands for horizontal alignment

# Adjust layout
plt.tight_layout()
fig.subplots_adjust(bottom=0.15)  # Make room for the legend

plt.savefig(model_folder/'trends_treatment_cam_only_aos_cost.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cam_only', data=df_treated_filtered, marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Expenditures CM - MHI  (CHF)')
axs[0, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cam_only', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Expenditures CM - MHI  (CHF)')
axs[0, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cam_only', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Expenditures CM - MHI  (CHF)')
axs[1, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cam_only', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Expenditures CM - MHI (CHF)')
axs[1, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_cam_only_aos_cost.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
df_spenders_cam_lca = df_treated_filtered[df_treated_filtered.treatment_cam_only == 1]

In [ ]:
df_treated_filtered[df_treated_filtered.treatment_cumulated == 1].uuid.nunique()

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 7))

# First subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_LCA', hue='treatment_cam_only', data=df_treated_filtered, marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Average Spending on Complementary Medicine')
axs[0, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_LCA', hue='treatment_cam_only', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Average Spending on Complementary Medicine')
axs[0, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_LCA', hue='treatment_cam_only', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Average Spending on Complementary Medicine')
axs[1, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_LCA', hue='treatment_cam_only', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Average Spending on Complementary Medicine')
axs[1, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_cam_only_lca_cost.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
df_treated_filtered['treatment'].sort_values().unique()


In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# List of dataframes and titles
dataframes = [df_treated_filtered, df_healthy, df_multimorbid, df_cancer]
titles = ['All individuals', 'Individuals without chronic conditions', 'Multimorbid individuals', 'Individuals with cancer']

# Create subplots
for i, (ax, df, title) in enumerate(zip(axs.flat, dataframes, titles)):
    sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment', data=df, marker='o', ax=ax, legend=False)
    ax.set_title(title)
    ax.set_xlabel('Year')
    ax.set_ylabel('Conventional Medicine\nexpenditures (CHF)')
    ax.grid(True)

# Manually create legend handles and labels
legend_handles = [plt.Line2D([0], [0], marker='o', color=palette[i], 
                             label=dic_labels[treatment], markersize=8, linestyle='-') 
                  for i, treatment in enumerate(df_treated_filtered['treatment'].sort_values().unique())]

# Create a single legend with the correct title
legend = fig.legend(handles=legend_handles, 
           title='Complementary/Alternative Medicine (CAM)\nthrough Supplementary Insurance (SI)', 
           loc='lower center', bbox_to_anchor=(0.5, -0.04), ncol=2)
plt.setp(legend.get_title(), ha='center')  # 'ha' stands for horizontal alignment

# Adjust layout
plt.tight_layout()
fig.subplots_adjust(bottom=0.15)  # Make room for the legend

plt.savefig(model_folder/'trends_treatment_aos_cost.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment', data=df_treated_filtered, marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Expenditures CM - MHI (CHF)')
axs[0, 0].legend(title='CAM - SI usage', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Expenditures CM - MHI (CHF)')
axs[0, 1].legend(title='CAM - SI usage', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Expenditures CM - MHI (CHF)')
axs[1, 0].legend(title='CAM - SI usage', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Expenditures CM - MHI (CHF)')
axs[1, 1].legend(title='CAM - SI usage', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_aos_cost.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.lineplot(x='NOANNEE', y='cds', hue='treatment', data=df_treated_filtered, marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Chronic disease score (CDS)')
axs[0, 0].legend(title='CAM - SI usage', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='cds', hue='treatment', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Chronic disease score (CDS)')
axs[0, 1].legend(title='Treatment', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='NOANNEE', y='cds', hue='treatment', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Chronic disease score (CDS)')
axs[1, 0].legend(title='CAM - SI usage', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='NOANNEE', y='cds', hue='treatment', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Chronic disease score (CDS)')
axs[1, 1].legend(title='CAM - SI usage', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_cds.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# List of dataframes and titles
dataframes = [df_treated_filtered, df_healthy, df_multimorbid, df_cancer]
titles = ['All', 'No-PCG individuals', 'Multimorbid patients', 'Patients with cancer']

# Create subplots
for i, (ax, df, title) in enumerate(zip(axs.flat, dataframes, titles)):
    sns.lineplot(x='NOANNEE', y='n_atc', hue='treatment', data=df, marker='o', ax=ax, legend=False)
    ax.set_title(title)
    ax.set_xlabel('Year')
    ax.set_ylabel('Number of distinct therapeutic subgroups')
    ax.grid(True)

# Manually create legend handles and labels
legend_handles = [plt.Line2D([0], [0], marker='o', color=palette[i], 
                             label=dic_labels[treatment], markersize=8, linestyle='-') 
                  for i, treatment in enumerate(df_treated_filtered['treatment'].unique())]

# Create a single legend with the correct title
fig.legend(handles=legend_handles, 
           title='Complementary/Alternative Medicine usage through Supplementary Insurance', 
           loc='lower center', bbox_to_anchor=(0.5, -0.04), ncol=2)
# Adjust layout
plt.tight_layout()
fig.subplots_adjust(bottom=0.15)  # Make room for the legend

plt.savefig(model_folder/'trends_treatment_atcs.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))
# First subplot
sns.lineplot(x='NOANNEE', y='n_atc', hue='treatment_cam_only', data=df_treated_filtered[df_treated_filtered.PRESTATIONS_BRUTES_AOS > df_treated_filtered.MTFRANCHISECOUV], marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Number of ATCs')
axs[0, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='n_atc', hue='treatment_cam_only', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Number of ATCs')
axs[0, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='NOANNEE', y='n_atc', hue='treatment_cam_only', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Number of ATCs')
axs[1, 0].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='NOANNEE', y='n_atc', hue='treatment_cam_only', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Number of ATCs')
axs[1, 1].legend(title='CAM - MHI usage', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_atcs_MHI.png', dpi = 300, bbox_inches = 'tight')


In [ ]:
# Create a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(9, 6))

# First subplot
sns.lineplot(x='year', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cumulated', data=df_treated_filtered, marker='o', ax=axs[0, 0])
axs[0, 0].set_title('All')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Average Number of ATCs')
axs[0, 0].legend(title='Treatment', loc='upper left')
axs[0, 0].grid(True)
# Second subplot
sns.lineplot(x='NOANNEE', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cumulated', data=df_healthy, marker='o', ax=axs[0, 1])
axs[0, 1].set_title('No-PCG individuals')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Average Number of ATCs')
axs[0, 1].legend(title='Treatment', loc='upper left')
axs[0, 1].grid(True)

# Third subplot
# Create the line plot
sns.lineplot(x='year', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cumulated', data=df_multimorbid, marker='o', ax=axs[1, 0])
axs[1, 0].set_title('Multimorbid patients')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Average Number of ATCs')
axs[1, 0].legend(title='Treatment', loc='upper left')
axs[1, 0].grid(True)

# Fourth subplot
sns.lineplot(x='year', y='PRESTATIONS_BRUTES_AOS', hue='treatment_cumulated', data=df_cancer, marker='o', ax=axs[1, 1])

axs[1, 1].set_title('Patients with cancer')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Average Number of ATCs')
axs[1, 1].legend(title='Treatment', loc='upper left')
axs[1, 1].grid(True)

# Adjust layout
plt.tight_layout()
plt.savefig(model_folder/'trends_treatment_cumulated_aos_cost.png', dpi = 300, bbox_inches = 'tight')

## Cancer patients

In [ ]:
df_treated_filtered.groupby('Cancer_PCG')['treatment_lca_cam'].mean()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 4))

g = sns.barplot(data=df_treated_filtered, y="treatment", x='Cancer_PCG', width=.6, palette="vlag", ax=axs[0])
g.set_ylabel('Prevalence of CAM use (SI)')

g = sns.barplot(data=df_treated_filtered, y="treatment_cam_only", x='Cancer_PCG', width=.6, palette="vlag", ax=axs[1])
g.set_ylabel('Prevalence of CAM use (MHI)')

g = sns.barplot(data=df_treated_filtered, y="treatment_lca_cam", x='Cancer_PCG', width=.6, palette="vlag", ax=axs[2])
g.set_ylabel('Prevalence of CAM use (SI+MHI)')

plt.savefig(results_folder/'Figures_rapport/Prevalence_cancer_patients.png', dpi = 300, bbox_inches='tight')

In [ ]:
df_treated_filtered.filter(regex='Meth.*_amount$|Cancer')

In [ ]:

# Melt the dataframe to long format for methods columns.
methods_regex = 'Methodes.*_amount$'
methods_long_df = df_treated_filtered.melt(id_vars='Cancer_PCG', 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Method', value_name='Amount')


In [ ]:
methods_long_df['Method'] = methods_long_df['Method'].str.replace('_amount','')

In [ ]:
# Now use Seaborn to create a bar plot with error bars.
plt.figure(figsize=(14, 7))
sns.barplot(data=methods_long_df, x='Method', y='Amount', hue='Cancer_PCG', errorbar='ci')

# Improve the aesthetics and readability.
# plt.title('Comparison of Method Amounts by Cancer Classification with Error Bars')
plt.xlabel('Methods')
plt.ylabel('Average yearly CAM claims amount (SI) (CHF)')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability.
# plt.legend(title='Cancer_PCG')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels.
plt.savefig(results_folder/'Figures_rapport/Costs_by_lca_method_cancer_patients.png', dpi = 300, bbox_inches='tight')

In [ ]:
for col in df_treated_filtered.filter(regex=methods_regex).columns:
    df_treated_filtered[f'{col}_binary'] = df_treated_filtered[col].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
methods_long_df

In [ ]:
methods_regex = 'Methodes.*_binary$'
methods_long_df = df_treated_filtered.melt(id_vars='Pain_PCG', 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Method', value_name='Prevalence')
methods_long_df['Method'] = methods_long_df['Method'].str.replace('_amount_binary','')

In [ ]:
# Now use Seaborn to create a bar plot with error bars.
plt.figure(figsize=(14, 7))
sns.barplot(data=methods_long_df, x='Method', y='Prevalence', hue='Pain_PCG', errorbar='ci')

# Improve the aesthetics and readability.
# plt.title('Comparison of Method Amounts by Cancer Classification with Error Bars')
plt.xlabel('Methods')
plt.ylabel('Prevalence')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability.
# plt.legend(title='Cancer_PCG')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels.
plt.savefig(results_folder/'Figures_rapport/Prevalence_by_lca_method_pain_patients.png', dpi = 300, bbox_inches='tight')

In [ ]:
methods_regex = 'Methodes.*_binary$'
methods_long_df = df_treated_filtered.melt(id_vars='Cancer_PCG', 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Method', value_name='Prevalence')
methods_long_df['Method'] = methods_long_df['Method'].str.replace('_amount_binary','')

In [ ]:
# Now use Seaborn to create a bar plot with error bars.
plt.figure(figsize=(14, 7))
sns.barplot(data=methods_long_df, x='Method', y='Prevalence', hue='Cancer_PCG', errorbar='ci')

# Improve the aesthetics and readability.
# plt.title('Comparison of Method Amounts by Cancer Classification with Error Bars')
plt.xlabel('Methods')
plt.ylabel('Prevalence')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability.
# plt.legend(title='Cancer_PCG')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels.
plt.savefig(results_folder/'Figures_rapport/Prevalence_by_lca_method_cancer_patients.png', dpi = 300, bbox_inches='tight')

### Prevalence utilisation des CAM par PCG

In [ ]:
methods_regex = 'PCG'
methods_long_df = df_treated_filtered.melt(id_vars='treatment_cam_only', 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Disease', value_name='Prevalence')
methods_long_df['Disease'] = methods_long_df['Disease'].str.replace('_PCG','')

methods_long_df = methods_long_df.rename(columns = {"treatment_cam_only":'Treatment 3 (MHI only)'})

plt.figure(figsize=(14, 7))
sns.barplot(data=methods_long_df, x='Disease', y='Prevalence', hue='Treatment 3 (MHI only)', errorbar='ci')

# Improve the aesthetics and readability.
# plt.title('Comparison of Method Amounts by Cancer Classification with Error Bars')
plt.xlabel('Methods')
plt.ylabel('Prevalence')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability.
# plt.legend(title='Cancer_PCG')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels.
plt.savefig(results_folder/'Figures_rapport/Prevalence_cam_only_by_pcg.png', dpi = 300, bbox_inches='tight')

In [ ]:
methods_regex = 'PCG'
methods_long_df = df_treated_filtered.melt(id_vars='treatment', 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Disease', value_name='Prevalence')
methods_long_df['Disease'] = methods_long_df['Disease'].str.replace('_PCG','')

methods_long_df = methods_long_df.rename(columns = {"treatment":'Treatment 1 (SI only)'})

plt.figure(figsize=(14, 7))
sns.barplot(data=methods_long_df, x='Disease', y='Prevalence', hue='Treatment 1 (SI only)', errorbar='ci')

# Improve the aesthetics and readability.
# plt.title('Comparison of Method Amounts by Cancer Classification with Error Bars')
plt.xlabel('Methods')
plt.ylabel('Prevalence')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability.
# plt.legend(title='Cancer_PCG')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels.
plt.savefig(results_folder/'Figures_rapport/Prevalence_lca_only_by_pcg.png', dpi = 300, bbox_inches='tight')

## HEATMAP PCG VS METHODS

In [ ]:
methods_regex = 'Methodes.*_binary$'
# methods_long_df = df_treated_filtered.melt(id_vars='Cancer_PCG', 
#                           value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
#                           var_name='Method', value_name='Prevalence')
df_treated_filtered.filter(regex=methods_regex).columns

In [ ]:
methods_regex = 'PCG'
methods_long_df = df_treated_filtered.melt(id_vars=['uuid','year'], 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Disease', value_name='Prevalence')
methods_long_df['Disease'] = methods_long_df['Disease'].str.replace('_PCG','')


In [ ]:
methods_long_df = methods_long_df[methods_long_df.Prevalence == 1]

df_test = df_treated_filtered.filter(regex='Methodes.*binary$|uuid|year')
# g = sns.clustermap(df_test, method="average", cmap="coolwarm", standard_scale=1)

df_disease_methods = pd.merge(methods_long_df,df_test, on = ['uuid','year'])

In [ ]:
df_disease_methods[df_disease_methods['Disease'] == 'Asthma']['Methodes ayurvediques_amount_binary'].sum()

In [ ]:
df_disease_methods[df_disease_methods['Disease'] == 'Cancer']['Methodes ayurvediques_amount_binary'].sum()

In [ ]:
df_disease_methods = df_disease_methods.groupby('Disease').sum()

In [ ]:
exclude_columns = ['year','uuid','Prevalence']

for col in df_disease_methods.columns:
    if col not in exclude_columns:
        print(col)
        df_disease_methods[col] = df_disease_methods[col] / df_disease_methods['Prevalence']

In [ ]:
df_disease_methods = df_disease_methods.drop(['year','Prevalence'], axis = 1)

df_disease_methods.columns = df_disease_methods.columns.str.replace('_amount_binary','')

In [ ]:
# # plt.figure(figsize=(32, 4))
# sns.clustermap(df_disease_methods, z_score = 1, cmap = 'coolwarm', figsize=(8, 8))
# plt.title('Typology based on therapy Methods and PCGs')
# plt.savefig(results_folder/'Figures_rapport/Prevalence_methods_vs_pcg.png', dpi = 300, bbox_inches='tight')

In [ ]:
method_mapping = {
    'Methodes ayurvediques': 'Ayurvedic methods',
    "Methodes d'art therapie": 'Art therapy methods',
    'Methodes de massage': 'Massage methods',
    'Methodes energetiques': 'Energetic methods',
    'Methodes energetiques manuelles': 'Manual energetic methods',
    'Methodes hydrotherapeutiques': 'Hydrotherapeutic methods',
    'Methodes occidentales': 'Western methods',
    'Methodes orientales': 'Oriental methods',
    'Methodes prescriptives': 'Prescriptive methods',
    'Methodes psychologiques complementaires': 'Complementary psychological methods',
    'Methodes reflexes': 'Reflex methods',
    'Methodes therapeutiques par le mouvement': 'Movement-based therapeutic methods'
}

disease_mapping = {

    'Asthma' : "Asthma",
  'Diabetes' : "Diabetes",
  'Cancer'   : "Cancer",
  'Epilepsy' : "Epilepsy",
  'Glaucoma' : "Glaucoma",
  'HIV_AIDS' : "HIV/AIDS",
  'Heart_disease' : "Heart Disease",
  'Hypertension_related' : "Hypertension",
  'Immune' : "Immune Disorders",
  'Inflammatory' : "Inflammatory Disorders",
  'Mental' : "Mental Health Conditions",
  'Other' : "Other Conditions",
  'Pain' : "Pain Related Conditions",
  'Parkinson' : "Parkinson's Disease",
  'Thyroid' : "Thyroid Disorders",
}

In [ ]:
df_disease_methods = df_disease_methods.rename(columns=method_mapping)
df_disease_methods = df_disease_methods.rename(index=disease_mapping)

In [ ]:
# Better color bar handling
cmap = sns.diverging_palette(220, 20, as_cmap=True)  # Create a custom diverging palette

# Generate the clustermap
g = sns.clustermap(
    df_disease_methods.drop('uuid',axis=1),
    z_score=1,
    cmap=cmap,
    figsize=(10, 10),  # Adjust size to fit labels if needed
    linewidths=.5,  # Add lines between cells to enhance readability
    cbar_kws={'label': 'Standardized Scores'}  # Label for the color bar
)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)  # Rotate the y labels for better readability
# plt.set_xlabel('CAM - SI methods')
# plt.setp(g.ax_heatmap.get_xticklabels(), rotation=45)  # Rotate the x labels for readability
# g.ax_heatmap.set_title('Typology based on Therapy Methods and PCGs', pad=-10)  # Set title with padding for visibility
plt.savefig(results_folder/'Figures_rapport/Heatmap_disease_SI_methods.png', dpi = 300, bbox_inches='tight')

In [ ]:
df_cdposition = pd.read_parquet('/Users/david/Dropbox/PhD/GitHub/SanteIntegra/Data/processed/Intermediate datasets/df_amount_by_cdposition.parquet.gzip')

In [ ]:
columns_to_update = ['Acupuncture', 'Anthroposophic medicine', 'Homeopathy', 'Neural therapy', 'Phytotherapy', 'Traditional Chinese medicine']
df_treated_filtered[[i+'_b' for i in columns_to_update]] = df_treated_filtered[columns_to_update].apply(lambda x: np.where(x > 0, 1, x))

In [ ]:
'|'.join(df_cdposition.filter(regex='_b').columns.tolist())

In [ ]:
methods_regex = 'Acupuncture_b|Anthroposophic medicine_b|Homeopathy_b|Neural therapy_b|Phytotherapy_b|Traditional Chinese medicine_b'
# methods_long_df = df_treated_filtered.melt(id_vars='Cancer_PCG', 
#                           value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
#                           var_name='Method', value_name='Prevalence')
df_treated_filtered.filter(regex=methods_regex).columns

In [ ]:
methods_regex = 'PCG'
methods_long_df = df_treated_filtered.melt(id_vars=['uuid','year'], 
                          value_vars=df_treated_filtered.filter(regex=methods_regex).columns, 
                          var_name='Disease', value_name='Prevalence')
methods_long_df['Disease'] = methods_long_df['Disease'].str.replace('_PCG','')


In [ ]:
methods_long_df = methods_long_df[methods_long_df.Prevalence == 1]

df_test = df_treated_filtered.filter(regex='uuid|year|Acupuncture_b|Anthroposophic medicine_b|Homeopathy_b|Neural therapy_b|Phytotherapy_b|Traditional Chinese medicine_b')
# g = sns.clustermap(df_test, method="average", cmap="coolwarm", standard_scale=1)

df_disease_methods = pd.merge(methods_long_df,df_test, on = ['uuid','year'])

In [ ]:
df_disease_methods = df_disease_methods.groupby('Disease').sum()

In [ ]:
exclude_columns = ['uuid','year','Prevalence']

for col in df_disease_methods.columns:
    if col not in exclude_columns:
        df_disease_methods[col] = df_disease_methods[col] / df_disease_methods['Prevalence']

In [ ]:
df_disease_methods = df_disease_methods.drop(['year','uuid','Prevalence'], axis = 1)
df_disease_methods.columns = df_disease_methods.columns.str.replace('_b','')

In [ ]:
# df_disease_methods = df_disease_methods.rename(columns=method_mapping)
df_disease_methods = df_disease_methods.rename(index=disease_mapping)

In [ ]:
# Generate the clustermap
g = sns.clustermap(
    df_disease_methods,
    z_score=1,
    cmap=cmap,
    figsize=(10, 10),  # Adjust size to fit labels if needed
    linewidths=.5,  # Add lines between cells to enhance readability
    cbar_kws={'label': 'Standardized Scores'}  # Label for the color bar
)
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0) 
plt.savefig(results_folder/'Figures_rapport/Heatmap_disease_MHI_methods.png', dpi = 300, bbox_inches='tight')

In [ ]:
# # plt.figure(figsize=(32, 4))
# sns.clustermap(methods_long_df, z_score = 1, cmap = 'coolwarm', figsize=(25, 12))
# plt.title('Typology of communities based on Therapies')
# plt.xlabel('Méthodes thérapeutiques')
# plt.ylabel('Community')
# plt.show()

## Models on subgroups

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
def run_mixedlm_ri(df, formula, n_formula, sample=False):
    print('Starting Mixed effect modeling with Random Intercepts for UUID')
    me_intercept_model = smf.mixedlm(formula, data=df, groups=df['uuid']).fit(reml=False)
    if sample:
        pass
    else:
        output_model(me_intercept_model, n_formula, 'RI')
    print(me_intercept_model.summary())
    # Obtain the residuals
    residuals = me_intercept_model.resid
    # Obtain the fitted values
    fitted_values = me_intercept_model.fittedvalues
    # Obtain the random intercepts
    random_intercepts = pd.DataFrame({k: v['Group'] for k, v in  me_intercept_model.random_effects.items()}, index=['random_intercept']).T
    
    # Store the residuals in the original dataframe
    df[f'model{n_formula}_ri_residuals'] = residuals
    # Store the fitted values in the original dataframe
    df[f'model{n_formula}_ri_fitted_values'] = fitted_values
    # Store the random intercepts
    df[f'model{n_formula}_ri_random_intercepts'] = df['uuid'].map(random_intercepts['random_intercept'].to_dict())
    # Calculate linear predictor
    model1_ri_lin_pred = fitted_values - df[f'model{n_formula}_ri_random_intercepts'] # the linear predictor

In [ ]:
def run_mixedlm_rs(df, formula, n_formula, sample=False):
    print('Starting Mixed effect modeling with Random Intercepts for UUID and Random Slopes for YEAR')
    me_slope_model = smf.mixedlm(formula, data=df, 
                              groups=df['uuid'], 
                              re_formula="~year").fit(reml=False)  
    if sample:
        pass
    else:
        output_model(me_slope_model, n_formula, 'RS')
    print(me_slope_model.summary())
    # Obtain the residuals
    residuals = me_slope_model.resid
    # Obtain the fitted values
    fitted_values = me_slope_model.fittedvalues
    # Obtain the random intercepts
    random_intercepts = pd.DataFrame({k: v['Group'] for k, v in  me_slope_model.random_effects.items()}, index=['random_intercept']).T
    # Store the residuals in the original dataframe
    df[f'model{n_formula}_rs_residuals'] = residuals
    # Store the fitted values in the original dataframe
    df[f'model{n_formula}_rs_fitted_values'] = fitted_values
    # Store the random intercepts
    df[f'model{n_formula}_rs_random_intercepts'] = df['uuid'].map(random_intercepts['random_intercept'].to_dict())
    # Calculate linear predictor
    model1_ri_lin_pred = fitted_values - df[f'model{n_formula}_rs_random_intercepts'] # the linear predictor
    return df, me_slope_model

In [ ]:
def run_mixedlm_rs_complex(df, formula, n_formula, sample=False):
    print('Starting Mixed effect modeling with Random Intercepts for UUID and CANTON and Random Slopes for YEAR')
    # Specify the random intercepts and slopes for 'code' and random intercepts for 'famcode'
    me_slope_2_model = smf.mixedlm(formula, df, 
                          groups=df['uuid'], 
                          re_formula='1 + year', 
                          vc_formula={'CANTON_ACRONYM_first': '0 + C(CANTON_ACRONYM_first)'}).fit(reml=False)
    if sample:
        pass
    else:
        output_model(me_slope_2_model, n_formula, 'RS2')

    print(me_slope_2_model.summary())
    # Obtain the residuals
    residuals = me_slope_2_model.resid
    # Obtain the fitted values
    fitted_values = me_slope_2_model.fittedvalues
    # Obtain the random intercepts
    random_intercepts = pd.DataFrame({k: v['Group'] for k, v in  me_slope_2_model.random_effects.items()}, index=['random_intercept']).T
    
    # Store the residuals in the original dataframe
    df[f'model{n_formula}_rs_complex_residuals'] = residuals
    # Store the fitted values in the original dataframe
    df[f'model{n_formula}_rs_complex_fitted_values'] = fitted_values
    # Store the random intercepts
    df[f'model{n_formula}_rs_complex_random_intercepts'] = df['uuid'].map(random_intercepts['random_intercept'].to_dict())
    return df, me_slope_2_model

In [ ]:
def run_mixedlm_ri_complex(df, formula, n_formula, sample=False):
    print('Starting Mixed effect modeling with Random Intercepts for UUID and for CANTON')
    me_intercept2_model = smf.mixedlm(formula, df, 
                          groups=df['uuid'], 
                          re_formula='1', 
                          vc_formula={'CANTON_ACRONYM_first': '0 + C(CANTON_ACRONYM_first)'}).fit(reml=False)   
    if sample:
        pass
    else:
        output_model(me_intercept2_model, n_formula, 'RI2')
    print(me_intercept2_model.summary())
    # Obtain the residuals
    residuals = me_intercept2_model.resid
    # Obtain the fitted values
    fitted_values = me_intercept2_model.fittedvalues
    # Obtain the random intercepts
    random_intercepts = pd.DataFrame({k: v['Group'] for k, v in  me_intercept2_model.random_effects.items()}, index=['random_intercept']).T
    
    # Store the residuals in the original dataframe
    df[f'model{n_formula}_ri_complex_residuals'] = residuals
    # Store the fitted values in the original dataframe
    df[f'model{n_formula}_ri_complex_fitted_values'] = fitted_values
    # Store the random intercepts
    df[f'model{n_formula}_ri_complex_random_intercepts'] = df['uuid'].map(random_intercepts['random_intercept'].to_dict())
    # Calculate linear predictor
    model1_ri_lin_pred = fitted_values - df[f'model{n_formula}_ri_complex_random_intercepts'] # the linear predictor
    return df, me_intercept2_model

In [ ]:
from statsmodels.graphics.gofplots import qqplot

def plot_model_diagnostics(df, model, model_type, fitted_values, residuals, random_intercepts, figsize=(10, 6)):
    """
    Plots diagnostics for a fitted regression model.

    Parameters:
    fitted_values -- an array-like object of fitted values from the model
    residuals -- an array-like object of residuals from the model
    figsize -- a tuple defining the figure size for the plots
    """
    model_n_folder = model_folder/model
    model_n_type_folder = model_n_folder/model_type

    if not os.path.exists(model_n_folder):
        os.makedirs(model_n_folder)
    if not os.path.exists(model_n_type_folder):
        os.makedirs(model_n_type_folder)
    # Residuals vs Fitted plot
    plt.figure(figsize=figsize)
    plt.scatter(fitted_values, residuals, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Fitted Values')
    plt.ylabel('Residuals')
    plt.title('Residuals vs Fitted')
    plt.savefig(model_n_type_folder/'residuals_vs_fitted.png', dpi = 300, bbox_inches = 'tight')
    
    # Residuals across years
    plt.figure(figsize=figsize)
    sns.stripplot(x='year', y=residuals, data=df, jitter=True, alpha = 0.5)
    sns.despine()
    plt.savefig(model_n_type_folder/'res_over_years.png', dpi = 300, bbox_inches = 'tight')
    
    # QQ plot for level 1 residuals
    fig,ax = plt.subplots(figsize = (10,5))
    qqplot(residuals, line='s', ax = ax, markersize=0.5)
    plt.title('QQ Plot of level 1 Residuals')
    plt.savefig(model_n_type_folder/'qqplot_lvl1.png', dpi = 300, bbox_inches = 'tight')
    # QQ plot for level 2 residuals
    fig,ax = plt.subplots(figsize = (10,5))
    qqplot(random_intercepts, line='s', ax = ax, markersize=0.5)
    plt.title('QQ Plot of level 2 Residuals')
    plt.savefig(model_n_type_folder/'qqplot_lvl2.png', dpi = 300, bbox_inches = 'tight')

    # Histogram of residuals
    plt.figure(figsize=figsize)
    sns.histplot(residuals, kde=True)
    plt.xlabel('Residuals')
    plt.title('Histogram of Residuals')
    plt.savefig(model_n_type_folder/'residuals_hist.png', dpi = 300, bbox_inches = 'tight')

    # Scale-Location plot (Absolute Residuals vs Fitted Values)
    plt.figure(figsize=figsize)
    plt.scatter(fitted_values, np.abs(residuals), alpha=0.5)
    plt.axhline(y=np.mean(np.abs(residuals)), color='r', linestyle='--')
    plt.xlabel('Fitted Values')
    plt.ylabel('Absolute Residuals')
    plt.title('Scale-Location Plot')
    plt.savefig(model_n_type_folder/'abs_residuals_vs_fitted.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
def output_model(model, model_n, model_type):
    print("Outputting the model")
    model_n_folder = model_folder/f"Model - {model_n}"
    model_n_type_folder = model_n_folder/model_type
    if not os.path.exists(model_n_folder):
        os.makedirs(model_n_folder)
    if not os.path.exists(model_n_type_folder):
        os.makedirs(model_n_type_folder)
    # Convert summary to dataframe
    quality_metrics = pd.DataFrame(model.summary().tables[0])
    parameters = pd.DataFrame(model.summary().tables[1])
    if model_type == 'OLS':
        diagnostics = pd.DataFrame(model.summary().tables[2])
        diagnostics.to_csv(model_n_type_folder/'model_diagnostics.csv')
    else:
        quality_metrics.loc[6] = ['AIC',model.aic,'','']
        quality_metrics.loc[7] = ['BIC',model.bic,'','']
    # Save to CSV
    quality_metrics.to_csv(model_n_type_folder/'model_metrics.csv')
    parameters.to_csv(model_n_type_folder/'model_parameters.csv')

## Run models

### All data - Model 1

In [ ]:
# model_formula = 'ihs_cost_aos ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
# df1_all, me_slope_model1_all = run_mixedlm_rs(df_treated_filtered, model_formula, "All", False)

In [ ]:
# plot_model_diagnostics(df1_all, 'Model - All','RS', df1_all['modelAll_rs_fitted_values'], df1_all['modelAll_rs_residuals'], df1_all['modelAll_rs_random_intercepts'])

### All data - Model 2 

In [ ]:
# model_formula = 'ihs_cost_aos_ambu ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
# df2_all, me_slope_model2_all = run_mixedlm_rs(df_treated_filtered, model_formula, "All - Ambulatoire", True)

### All data - Model 3

In [ ]:
# model_formula = 'ihs_cost_aos_stat ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
# df3_all, me_slope_model3_all = run_mixedlm_rs(df_treated_filtered, model_formula, "All - Stationnaire", True)

### Individuals with multimorbidity - Model 1

In [ ]:
df_multimorbid.filter(regex='treatment')

In [ ]:
model_formula = 'ihs_cost_aos ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
df1_multi, me_slope_model1_multi = run_mixedlm_rs(df_multimorbid, model_formula, "Multimorbidity", False)

In [ ]:
# Extract variance components
var_fixed = me_slope_model1_multi.fe_params.var()  # Fixed effects variance
# var_random = me_slope_model1_multi.random_effects.var()  # Random effects variance
var_resid = me_slope_model1_multi.resid.var()  # Residual variance

# Total variance
total_var = var_fixed  + var_resid

# Marginal R^2
r2_marginal = var_fixed / total_var

# Conditional R^2
r2_conditional = (var_fixed) / total_var

In [ ]:
df1_multi[df1_multi.modelMultimorbidity_rs_residuals.abs() >3].filter(regex='uuid|PCG|ihs|age_group|NOANNEE|Metho|PRESTATIONS_BRUTES_AOS|PRESTATIONS_BRUTES_LCA|atc|res')

In [ ]:
df1_multi[df1_multi.uuid == '4d0c0121-d8d8-4683-a778-826f9b9f8ef2'].filter(regex='PCG|ihs|age_group|NOANNEE|Metho|PRESTATIONS_BRUTES_AOS|PRESTATIONS_BRUTES_LCA|atc|res')

In [ ]:
plot_model_diagnostics(df1_multi, 'Model - Multimorbidity','RS', df1_multi['modelMultimorbidity_rs_fitted_values'], df1_multi['modelMultimorbidity_rs_residuals'], df1_multi['modelMultimorbidity_rs_random_intercepts'])

In [ ]:
fig = plt.figure(figsize = (16, 9))

ax = sns.scatterplot(y = me_slope_model1_multi.resid, x = me_slope_model1_multi.fittedvalues)

ax.set_title("RVF Plot")
ax.set_xlabel("Fitted Values")
ax.set_ylabel("Residuals")

In [ ]:
# Extracting random effects
random_effects = me_slope_model1_multi.random_effects

# Convert to DataFrame for easier handling
random_effects_df = pd.DataFrame([dict(re) for re in random_effects.values()], index=random_effects.keys())

# Plotting the distribution of random intercepts and slopes (if applicable)
for column in random_effects_df.columns:
    plt.figure(figsize=(8, 4))
    plt.hist(random_effects_df[column], bins=30, edgecolor='black')
    plt.title(f'Distribution of Random Effect: {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()


In [ ]:
z_scores = np.abs(random_effects_df - random_effects_df.mean()) / random_effects_df.std()
outliers = z_scores > 3  # Arbitrary threshold, can be adjusted

# Displaying potential outliers
print("Potential Outliers in Random Effects:")
print(outliers.sum())

In [ ]:
df_multimorbid[df_multimorbid.uuid == '017b50e0-a5bf-4ee7-ac12-c3dec88d25fd'].filter(regex='PCG|ihs|age_group|NOANNEE|Metho|PRESTATIONS_BRUTES_AOS|PRESTATIONS_BRUTES_LCA|atc|res')

In [ ]:
model_formula = 'ihs_cost_aos ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
df1_multi_complex, me_slope_model1_multi_complex = run_mixedlm_rs_complex(df_multimorbid, model_formula, "Multimorbid", True)

### Individuals with multimorbidity - Model 2

In [ ]:
model_formula = 'ihs_cost_aos_ambu ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
df2_multi, me_slope_model2_multi = run_mixedlm_rs(df_multimorbid, model_formula, "Multimorbidity - Ambulatoire", True)

### Individuals with multimorbidity - Model 3

In [ ]:
model_formula = 'ihs_cost_aos_ambu ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
df3_multi, me_slope_model3_multi = run_mixedlm_rs(df_multimorbid, model_formula, "Multimorbidity - Stationnaire", True)

### Individuals with cancer - Model 1

In [ ]:
model_formula = 'ihs_cost_aos ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
df1_cancer, me_slope_model1_cancer = run_mixedlm_rs(df_cancer, model_formula, "Cancer", True)

### Individuals with cancer - Model 2

In [ ]:
model_formula = 'ihs_cost_aos_ambu ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
df2_cancer, me_slope_model2_cancer = run_mixedlm_rs(df_cancer, model_formula, "Cancer - Ambulatoire", True)

### Individuals with cancer - Model 3

In [ ]:
model_formula = 'ihs_cost_aos_stat ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
df3_cancer, me_slope_model3_cancer = run_mixedlm_rs(df_cancer, model_formula, "Cancer - Stationnaire", True)

### Indivuals with cancer - Model 4

In [ ]:
model_formula = 'ihs_cost_pain ~  treatment*year + SEX_F + NBAGE + ssep3 + CDLANGUE + D_MEDIC_B + n_flags + n_atc + locdrhosp + MTFRANCHISECOUV + mean_no2 + n_outpatient_hosp +  n_inpatient_hosp'
df1_cancer, me_slope_model1_cancer = run_mixedlm_rs(df_cancer, model_formula, "Cancer", True)

In [ ]:
df_uuid_gp = df_treated_filtered[['uuid','NOANNEE','treatment','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_AMBULATOIRE','PRESTATIONS_BRUTES_STATIONNAIRE']].groupby('uuid').sum()

In [ ]:
sns.boxplot(data=df_uuid_gp, x='treatment', y='PRESTATIONS_BRUTES_STATIONNAIRE', showfliers=False)

In [ ]:
# Create the line plot
plt.figure(figsize=(10, 6))
sns.lineplot(x='year', y='ihs_cost_aos', hue='treatment', data=df_treated_filtered, marker='o')

plt.title('Treatment Effect Over Time')
plt.xlabel('Year')
plt.ylabel('Average Spending on Conventional Medicine')
plt.legend(title='Treatment', loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
model_formula = 'ihs_cost_aos ~  treatment*year + SEX_F + NBAGE + ssep2 + CDLANGUE + D_MEDIC_B + n_flags + Cancer_PCG + Pain_PCG + n_outpatient_hosp +  n_inpatient_hosp'
df2, me_slope_model2 = run_mixedlm_rs(df_multimorbid, model_formula, "Multimorbid", True)

In [ ]:
df2[df2.uuid=='546cfa0f-1c2f-4692-962c-d733cb5b49b8'].filter(regex='PCG|NOANNEE|ihs|hosp|PRESTATIONS_BRUTES_AOS|PRESTATIONS_BRUTES_LCA|atc|flags|residuals')

In [ ]:
df_multimorbid_nooutliers = df_multimorbid[df_multimorbid.modelMultimorbid_rs_residuals.abs()< 3]

In [ ]:
df_multimorbid[df_multimorbid.modelMultimorbid_rs_residuals.abs() >3]

#### Cas d'une femme avec epilepsie qui ne consomme plus de médicaments à partir de 2020 et uniquement du LCA

In [ ]:
df_multimorbid[df_multimorbid.uuid == '546cfa0f-1c2f-4692-962c-d733cb5b49b8'].filter(regex='PCG|NOANNEE|PRESTATIONS_BRUTES_AOS|PRESTATIONS_BRUTES_LCA|atc')

In [ ]:
df_multimorbid[df_multimorbid.uuid == '546cfa0f-1c2f-4692-962c-d733cb5b49b8']

### Modelling without outliers

In [ ]:
model_formula = 'ihs_cost_aos ~ treatment*year + SEX_F + NBAGE + ssep2 + CDLANGUE + D_MEDIC_B'
me_slope_model = run_mixedlm_rs(df_multimorbid_nooutliers, model_formula, "Multimorbid", True)

In [ ]:
plot_model_diagnostics(df_multimorbid, 'Model - Multimorbidty','RS', df_multimorbid['modelMultimorbid_rs_fitted_values'], df_multimorbid['modelMultimorbid_rs_residuals'], df_multimorbid['modelMultimorbid_rs_random_intercepts'])

In [ ]:
plot_model_diagnostics(df_multimorbid_nooutliers, 'Model - Multimorbidty','RS', df_multimorbid['modelMultimorbid_rs_fitted_values'], df_multimorbid_nooutliers['modelMultimorbid_rs_residuals'], df_multimorbid_nooutliers['modelMultimorbid_rs_random_intercepts'])

In [ ]:
import scipy.stats as stats
fig = plt.figure(figsize = (16, 9))

ax = sns.distplot(me_slope_model.resid, hist = False, kde_kws = {"fill" : True, "lw": 1}, fit = stats.norm)

ax.set_title("KDE Plot of Model Residuals (Blue) and Normal Distribution (Black)")
ax.set_xlabel("Residuals")

In [ ]:
fig = plt.figure(figsize = (16, 9))
ax = fig.add_subplot(111)

sm.qqplot(me_slope_model.resid, dist = stats.norm, line = 's', ax = ax)

ax.set_title("Q-Q Plot")

In [ ]:
fig = plt.figure(figsize = (16, 9))

ax = sns.scatterplot(y = me_slope_model.resid, x = me_slope_model.fittedvalues)

ax.set_title("RVF Plot")
ax.set_xlabel("Fitted Values")
ax.set_ylabel("Residuals")

### Reproducing analysis by Radoslaw Panczak

In [ ]:
df = df_multimorbid[df_multimorbid.NBAGE >18].copy()
df = df[df['PRESTATIONS_BRUTES_AOS'] > df['MTFRANCHISECOUV']]

In [ ]:
df['MTFRANCHISECOUV'] = df['MTFRANCHISECOUV'].astype('category')

In [ ]:
base_formula = "ihs_cost_aos ~"
# Model 0: Unadjusted
model_0 = smf.mixedlm(base_formula + '1', 
                      data=df_treated_filtered, 
                      groups=df_treated_filtered['uuid']).fit(reml=False)


# Model 1: Unadjusted
model_1 = smf.mixedlm(base_formula + '1', 
                      data=df, 
                      groups=df['uuid'], 
                      re_formula='1+year', 
                      vc_formula={'CANTON_ACRONYM_first': '0 + CANTON_ACRONYM_first'}).fit(reml=False)

# Model 2: Adjusted for Age, sex
model_2 = smf.mixedlm(base_formula + 'treatment*year + NBAGE + SEX_F', 
                      data=df, 
                      groups=df['uuid'], 
                      re_formula='1+year', 
                      vc_formula={'CANTON_ACRONYM_first': '0 + CANTON_ACRONYM_first'}).fit(reml=False)

# Model 3: Adjusted for SES and access to HC
model_3 = smf.mixedlm(base_formula + 'treatment*year + NBAGE + SEX_F + ssep3 + D_MEDIC_B', 
                      data=df, 
                      groups=df['uuid'], 
                      re_formula='1+year', 
                      vc_formula={'CANTON_ACRONYM_first': '0 + CANTON_ACRONYM_first'}).fit(reml=False)

In [ ]:
# Model 4: Fully adjusted
model_4 = smf.mixedlm(base_formula + 'treatment_cam_only*year + SEX_F + NBAGE + DEDUCTIBLE_300 + DEDUCTIBLE_500 + DEDUCTIBLE_1000 + DEDUCTIBLE_1500 + DEDUCTIBLE_2000 + MODEL_MF + MODEL_HMO + MODEL_TEL + ssep3_q + region_DE + region_FR + region_IT + n_flags + n_month_inpatienthosp_log + locdrhosp + Asthma_PCG + Diabetes_PCG + Cancer_PCG + Epilepsy_PCG + Glaucoma_PCG + HIV_AIDS_PCG + Heart_disease_PCG + Hypertension_related_PCG + Immune_PCG + Inflammatory_PCG + Mental_PCG + Other_PCG + Pain_PCG + Parkinson_PCG + Thyroid_PCG  + mean_no2 + mean_ndvi + urb_Peri_urban + urb_Urban + D_MEDIC_B_log', 
                      data=df, 
                      groups=df['uuid'], 
                      re_formula='1', 
                      vc_formula={'CANTON_ACRONYM_first': '0 + CANTON_ACRONYM_first'}).fit(reml=False)

In [ ]:
model_2 = smf.mixedlm(base_formula + 'treatment_lca_cam*year + NBAGE + SEX_F', 
                      data=df, 
                      groups=df['uuid'], 
                      re_formula='1', 
                      vc_formula={'CANTON_ACRONYM_first': '0 + CANTON_ACRONYM_first'}).fit(reml=False)


In [ ]:
print(model_2.summary())

In [ ]:
df.filter(regex='month')

In [ ]:
# Obtain the residuals
residuals = model_4.resid
# Obtain the fitted values
fitted_values = model_4.fittedvalues

df['fitted_values'] = fitted_values
df['residuals'] = residuals

In [ ]:
ri_canton = pd.DataFrame({k: v[2] for k, v in  model_4.random_effects.items()}, index=['ri_group']).T.to_dict()['ri_group']
df['ri_canton'] = df['uuid'].map(ri_canton)

In [ ]:
df['intercept'] = model_4.params['Intercept']
df['intercept_ri_canton'] = df['intercept']+df['ri_canton']
df['exp_intercept_ri_canton'] = np.sinh(df['intercept_ri_canton'])

In [ ]:
df['intercept_sex_f'] = df['intercept'] + model_4.params['SEX_F']*df['SEX_F']
df['intercept_ssep3'] = df['intercept'] + model_4.params['ssep3_q']*df['ssep3_q']
df['intercept_treatment'] = df['intercept'] + model_4.params['treatment']*df['treatment']

df['exp_intercept_sex_f'] = np.sinh(df['intercept_sex_f'])
df['exp_intercept_ssep3'] = np.sinh(df['intercept_ssep3'])
df['exp_intercept_treatment'] = np.sinh(df['intercept_treatment'])

cost_ratio = df.groupby('treatment')['exp_intercept_treatment'].mean()/df['exp_intercept_treatment'].mean()
# cost_ratio = df.groupby('SEX_F')['intercept_sex_f'].mean()/df['exp_intercept_sex_f'].mean()

In [ ]:
cost_ratio

In [ ]:
1.046433/0.939225

In [ ]:
df['treatment:year'] = df['year'] * df['treatment']

In [ ]:
dict_params = {'CDLANGUE[T.EN]':'LANG_EN', 'CDLANGUE[T.FR]':"LANG_FR", 'CDLANGUE[T.IT]':"LANG_EN", 'Urbanicity_simple[T.Rural]':'urb_Rural',
               'Urbanicity_simple[T.Périurbain]':'urb_Périurbain','Urbanicity_simple[T.Urban]':'urb_Urban',
               'Language[T.RM]':'region_RM','Language[T.IT]':'region_IT','Language[T.FR]':'region_FR','Language[T.DE]':'region_DE',
       'MTFRANCHISECOUV[T.500.0]':"DEDUCTIBLE_500", 'MTFRANCHISECOUV[T.1000.0]':"DEDUCTIBLE_1000",
       'MTFRANCHISECOUV[T.1500.0]':"DEDUCTIBLE_1500", 'MTFRANCHISECOUV[T.2000.0]':"DEDUCTIBLE_2000",
       'MTFRANCHISECOUV[T.2500.0]':"DEDUCTIBLE_2500", 'treatment':'treatment', 'year':'year', 'treatment:year':'treatment:year',
       'NBAGE':'NBAGE', 'SEX_F':'SEX_F', 'ssep3':'ssep3', 'D_MEDIC_B':'D_MEDIC_B', 'n_flags':'n_flags', 'n_atc':'n_atc', 'locdrhosp':'locdrhosp',
       'mean_no2':'mean_no2', 'n_outpatient_hosp':'n_outpatient_hosp', 'n_inpatient_hosp':'n_inpatient_hosp'}

In [ ]:
# Initialize a dictionary to store cost ratios
cost_ratios = {}
df['intercept'] = model_4.params['Intercept']
# Loop over each parameter in the model (excluding the intercept)
for param in model_4.params.index[1:-4]:  # Skips the intercept
    print(param)
    # Calculate the adjusted intercept
    df[f'intercept_{param}'] = df['intercept'] + model_4.params[param] * df[dict_params[param]]
    
    # Apply np.sinh transformation
    df[f'exp_intercept_{param}'] = np.sinh(df[f'intercept_{param}'])
    
    # Calculate and store the cost ratio
    cost_ratios[param] = df.groupby(dict_params[param])[f'exp_intercept_{param}'].mean() / df[f'exp_intercept_{param}'].mean()


In [ ]:
fig,ax = plt.subplots(figsize=(15,6))
ax = sns.pointplot(
    data=df, x="CANTON_ACRONYM_first", y="ri_canton", order = ['UR', 'GL', 'AR', 'ZG', 'SG', 'NW', 'JU', 'VS', 'SO', 'SZ', 'LU', 'NE',
       'BE', 'FR', 'TI', 'BL', 'SH', 'AG', 'ZH', 'VD', 'TG', 'BS', 'GR', 'GE',
       'OW', 'AI'],
    capsize=.2, join=False, errorbar=('ci', 95), errwidth=0.6)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.grid(True)
# Show the plot
plt.show

In [ ]:
cost_ratio = (df.groupby('CANTON_ACRONYM_first').exp_intercept_ri_canton.mean()/df.exp_intercept_ri_canton.mean()).to_dict()

In [ ]:
cantons = gpd.read_file('/Users/david/Dropbox/PhD/GitHub/SanteIntegra/Data/raw/OFS/swissBOUNDARIES3D_1_5_LV95_LN02.gpkg', layer = 'tlm_kantonsgebiet')
cantons = cantons.to_crs(2056)
swiss_cantons = {
    "Aargau": "AG",
    "Appenzell Ausserrhoden": "AR",
    "Appenzell Innerrhoden": "AI",
    "Basel-Landschaft": "BL",
    "Basel-Stadt": "BS",
    "Bern": "BE",
    "Fribourg": "FR",
    "Genève": "GE",
    "Glarus": "GL",
    "Graubünden": "GR",
    "Jura": "JU",
    "Luzern": "LU",
    "Neuchâtel": "NE",
    "Nidwalden": "NW",
    "Obwalden": "OW",
    "St. Gallen": "SG",
    "Schaffhausen": "SH",
    "Schwyz": "SZ",
    "Solothurn": "SO",
    "Thurgau": "TG",
    "Ticino": "TI",
    "Uri": "UR",
    "Vaud": "VD",
    "Valais":"VS",
    "Zug": "ZG",
    "Zürich": "ZH"
}

cantons['CANTON_ACRONYM_manual'] = cantons['name'].map(swiss_cantons)
cantons['CANTON_NAME_manual'] = cantons['name']

In [ ]:
cantons['cost_ratio'] = cantons['CANTON_ACRONYM_manual'].map(cost_ratio)

In [ ]:
# Define custom breaks
breaks = [i * 0.1 for i in range(8,14)]  # Adjust the range as needed
breaks.insert(0,0.5)
breaks.append(1.8)
# Categorize data
cantons['cost_ratio_cat'] = pd.cut(cantons['cost_ratio'], breaks)

# Plot with custom categories
fig, ax = plt.subplots(1, 1)
cantons.plot(column='cost_ratio_cat', cmap='RdYlBu_r', legend=True, ax=ax)

# Show the plot
plt.show()

In [ ]:
# Obtain the residuals
residuals = model_2.resid
# Obtain the fitted values
fitted_values = model_2.fittedvalues
# Obtain the random intercepts
random_intercepts_group = pd.DataFrame({k: v['Group'] for k, v in  model_2.random_effects.items()}, index=['ri_group']).T
random_intercepts_year = pd.DataFrame({k: v['year'] for k, v in  model_2.random_effects.items()}, index=['ri_year']).T
# random_intercepts_canton = pd.DataFrame({k: v['CANTON_ACRONYM_first'] for k, v in  model_2.random_effects.items()}, index=['random_intercept']).T
random_intercepts_group

In [ ]:
# Compare model fit using AIC and BIC
model_comparison = {
    "Model 1": (model_1.aic, model_1.bic),
    "Model 2": (model_2.aic, model_2.bic),
    "Model 3": (model_3.aic, model_3.bic),
    "Model 4": (model_4.aic, model_4.bic)
}

model_metrics = pd.DataFrame.from_dict(model_comparison).T
model_metrics.columns = ['AIC','BIC']
model_metrics

## Two part model

### Part 2 - Continuous multilevel model

In [ ]:
df_treated_filtered_t_1.shape

In [ ]:
df = df_treated_filtered_t_1[df_treated_filtered_t_1.NBAGE>18]

In [ ]:
df.shape

In [ ]:
df = df[df.PRESTATIONS_BRUTES_AOS > df.MTFRANCHISECOUV]

In [ ]:
df = df.rename(columns = {'age_80+':'age_80plus'})

In [ ]:
df.shape

In [ ]:
uuids = df[['uuid']].drop_duplicates()
uuid_sample = uuids.sample(n = 50000, random_state=42)
df_sample = df[df.uuid.isin(uuid_sample.uuid)].reset_index(drop=True)

In [ ]:
df_sample.groupby('uuid').size().sort_values().value_counts()

In [ ]:
model_formula = 'ihs_cost_aos ~  treatment_lca_cam*year + ihs_cost_aos_t_1 + SEX_F + NBAGE + MODEL_MF + MODEL_HMO + MODEL_TEL + ssep3_q + region_DE + region_FR + region_IT + D_MEDIC_B_log + n_flags + n_atc_log*n_month_inpatienthosp_log + locdrhosp + Asthma_PCG + Cancer_PCG + Diabetes_PCG + Epilepsy_PCG + Glaucoma_PCG + HIV_AIDS_PCG + Heart_disease_PCG + Hypertension_related_PCG + Immune_PCG + Inflammatory_PCG + Mental_PCG + Other_PCG + Pain_PCG + Parkinson_PCG + Thyroid_PCG + DEDUCTIBLE_300 + DEDUCTIBLE_500 + DEDUCTIBLE_1000 + DEDUCTIBLE_1500 + DEDUCTIBLE_2000 + mean_no2 + mean_ndvi + mean_carnight + urb_Peri_urban + urb_Urban'
df_ri, me_intercept_complex_model = run_mixedlm_ri_complex(df, model_formula, "AOS costs", True)

In [ ]:
model_formula = 'ihs_cost_aos ~  treatment_lca_cam*year + ihs_cost_aos_t_1 + SEX_F + NBAGE + MODEL_MF + MODEL_HMO + MODEL_TEL + ssep3_q + region_DE + region_FR + region_IT + D_MEDIC_B + n_flags + n_atc_log + locdrhosp + Cancer_PCG  + DEDUCTIBLE_300 + DEDUCTIBLE_500 + DEDUCTIBLE_1000 + DEDUCTIBLE_1500 + DEDUCTIBLE_2000 + mean_no2 + mean_ndvi + mean_carnight + n_month_inpatienthosp_log + urb_Peri_urban + urb_Urban'
df_sample_ri, me_intercept_complex_model_sample = run_mixedlm_ri_complex(df_sample, model_formula, "AOS costs", True)

In [ ]:
# model_formula = 'ihs_cost_aos ~  treatment_lca_cam*year + ihs_cost_aos_t_1 + n_inpatient_hosp_t_1 + SEX_F + NBAGE + MODEL_MF + MODEL_HMO + MODEL_TEL + ssep3_q + region_DE + region_FR + region_IT + D_MEDIC_B + n_flags + n_atc_log + locdrhosp + Cancer_PCG  + DEDUCTIBLE_300 + DEDUCTIBLE_500 + DEDUCTIBLE_1000 + DEDUCTIBLE_1500 + DEDUCTIBLE_2000 + mean_no2 + mean_ndvi + mean_carnight + n_inpatient_hosp_log + urb_Peri_urban + urb_Urban'
# df_sample_rs, me_rs_complex_model_sample = run_mixedlm_rs_complex(df_sample, model_formula, "AOS costs", True)

In [ ]:
plot_model_diagnostics(df_ri, 'Model - AOS costs','RI - Complex', df_ri['modelAOS costs_ri_complex_fitted_values'], df_ri['modelAOS costs_ri_complex_residuals'], df_ri['modelAOS costs_ri_complex_random_intercepts'])

In [ ]:
fig,ax = plt.subplots(figsize = (10,5))
sm.qqplot(df_ri['modelAOS costs_ri_complex_residuals'], line='s', ax = ax, markersize=0.5)
plt.show()

In [ ]:
from scipy.stats import norm, uniform


In [ ]:
fig,ax = plt.subplots(figsize = (5,5))
sm.qqplot(df_ri['modelAOS costs_ri_complex_residuals'],norm, fit=True, line='45', ax = ax, markersize=0.5)
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize = (5,5))
sm.qqplot(df_ri['modelAOS costs_ri_complex_residuals'], line='s', ax = ax, markersize=0.5)
plt.show()

In [ ]:
# plot_model_diagnostics(df_sample_rs, 'Model - AOS costs','RS - Complex', df_sample_rs['modelAOS costs_rs_complex_fitted_values'], df_sample_rs['modelAOS costs_rs_complex_residuals'], df_sample_rs['modelAOS costs_rs_complex_random_intercepts'])

In linear mixed-effects models, the concept of R2 (coefficient of determination) is not as straightforward as in simple linear regression due to the presence of random effects. However, there are methods to calculate pseudo-R2 values that can give you an idea of the variance explained by the model. These pseudo-R2 values are typically categorized into two types: Marginal R2 and conditional R2.

Marginal $R^2$ (also known as $R_m^2$):
Represents the variance explained by the fixed effects alone.

Conditional $R^2$ (also known as $R_c^2$): Represents the variance explained by both the fixed and random effects.

**WARNING** This way of calculating var_random is incorrect and there is no easy way to compute it with statsmodels

In [ ]:
# Calculate the variance components
var_fixed = np.var(me_intercept_complex_model_sample.fittedvalues)
var_random = sum([np.var(random_effect) for random_effect in me_intercept_complex_model_sample.random_effects.values()])
var_resid = np.var(me_intercept_complex_model_sample.resid)

# Calculate marginal R^2
r2_marginal = var_fixed / (var_fixed + var_random + var_resid)

# Calculate conditional R^2
r2_conditional = (var_fixed + var_random) / (var_fixed + var_random + var_resid)

print(f"Marginal R^2: {r2_marginal}")
print(f"Conditional R^2: {r2_conditional}")

In [ ]:
df_ri[df_ri.uuid == '664deadf-902c-4bea-9e95-8e523d88c0f0'][['uuid','year','NBAGE','CDPHYSSEXE','CDLANGUE','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','modelAOS costs_ri_complex_residuals','n_inpatient_hosp','n_outpatient_hosp','ihs_cost_aos','ihs_cost_lca','modelAOS costs_ri_complex_fitted_values']]

In [ ]:
df_ri.groupby('n_month_inpatienthosp_log')['modelAOS costs_ri_complex_residuals'].mean().plot.bar()

In [ ]:

# Calculate the z-score for each student's height
z = np.abs(stats.zscore(df['ihs_cost_aos']))

# Identify outliers as students with a z-score greater than 3
threshold = 3
outliers = df[z > threshold]

In [ ]:
gp_outliers_wellmodeled = outliers[outliers['modelAOS costs_ri_complex_residuals'].between(-0.3,0.3)][['uuid','year','NBAGE','PRESTATIONS_BRUTES_AOS','n_atc','n_month_inpatienthosp']]

In [ ]:
gp_outliers_badmodeled = outliers[outliers['modelAOS costs_ri_complex_residuals'].between(-1,1) == False][['uuid','year','NBAGE','PRESTATIONS_BRUTES_AOS','n_atc','n_month_inpatienthosp']]

In [ ]:
gp_outliers_badmodeled.mean()

In [ ]:
gp_outliers_wellmodeled.mean()

In [ ]:
outliers.ihs_cost_aos.plot.hist(bins = 100)

In [ ]:
outliers.filter(regex='PCG|hosp')

In [ ]:
df[(df.locdrhosp == 1)&(df.n_month_inpatienthosp == 0)]

In [ ]:
outliers[outliers.PRESTATIONS_BRUTES_AOS < 100].shape[0]

In [ ]:
outliers.shape[0]

In [ ]:
df_sample.groupby('n_atc')['PRESTATIONS_BRUTES_AOS'].mean().plot.bar()

## Spatial variations

In [ ]:
# Obtain the residuals
residuals = model_0.resid
# Obtain the fitted values
fitted_values = model_0.fittedvalues

df_treated_filtered['model0_fitted_values'] = fitted_values
df_treated_filtered['model0_residuals'] = residuals

In [ ]:
base_formula = "ihs_cost_lca ~"
# Model 0: Unadjusted
model_0_lca = smf.mixedlm(base_formula + '1', 
                      data=df_treated_filtered, 
                      groups=df_treated_filtered['uuid']).fit(reml=False)

In [ ]:
# Obtain the residuals
residuals = model_0_lca.resid
# Obtain the fitted values
fitted_values = model_0_lca.fittedvalues

df_treated_filtered['model0_lca_fitted_values'] = fitted_values
df_treated_filtered['model0_lca_residuals'] = residuals

In [ ]:
from scipy import stats
fig = plt.figure(figsize = (16, 9))
ax = fig.add_subplot(111)

sm.qqplot(model_0_lca.resid, dist = stats.norm, line = 's', ax = ax)

ax.set_title("Q-Q Plot")

## Spatial analyses

In [ ]:
from scipy.spatial import cKDTree
from libpysal.weights.distance import get_points_array

In [ ]:
# df_spatial = df_treated_filtered[['uuid','NOANNEE','year','PRESTATIONS_BRUTES_AOS','PRESTATIONS_BRUTES_LCA','ihs_cost_aos','ihs_cost_lca','geometry']]

In [ ]:
weights = lps.weights.KNN(cKDTree(get_points_array(df_treated_filtered.geometry)), 128)

In [ ]:
import scipy.sparse
from libpysal.weights import WSP

In [ ]:
getis_values_1 = pyspace.compute_getis(df_treated_filtered, 'model0_residuals', weights, 999, transform_type='B', p_001=False)

In [ ]:
getis_values_2 = pyspace.compute_getis(df_treated_filtered, 'ihs_cost_aos', weights, 999, transform_type='B', p_001=False)

In [ ]:
getis_values_3 = pyspace.compute_getis(df_treated_filtered, 'model0_residuals', weights, 999, transform_type='B', p_001=False)

In [ ]:
fig, ax = pyspace.plotGetisMap(df_treated_filtered, "ihs_cost_aos_G_cl", markersize_s=0.0001, markersize_l=0.8, p_001=False, commune_name=False)

In [ ]:
fig, ax = pyspace.plotGetisMap(df_treated_filtered, "model0_residuals_G_cl", markersize_s=0.0001, markersize_l=0.8, p_001=False, commune_name=False)

In [ ]:
getis_values_1 = pyspace.compute_getis(df_treated_filtered, 'ihs_cost_lca', weights, 999, transform_type='B', p_001=False)

In [ ]:
getis_values_2 = pyspace.compute_getis(df_treated_filtered, 'model0_lca_residuals', weights, 999, transform_type='B', p_001=False)

In [ ]:
getis_values_3 = pyspace.compute_getis(df_treated_filtered, 'model0_lca_fitted_values', weights, 999, transform_type='B', p_001=False)

In [ ]:
fig, ax = pyspace.plotGetisMap(df_treated_filtered, "ihs_cost_lca_G_cl", markersize_s=0.01, markersize_l=0.1, p_001=False, commune_name=False)

In [ ]:
fig, ax = pyspace.plotGetisMap(df_treated_filtered, "model0_lca_residuals_G_cl", markersize_s=0.0001, markersize_l=0.8, p_001=False, commune_name=False)

In [ ]:
fig, ax = pyspace.plotGetisMap(df_treated_filtered, "model0_lca_fitted_values_G_cl", markersize_s=0.01, markersize_l=0.1, p_001=False, commune_name=False)

In [ ]:
fig, ax = pyspace.plot_getis_by_class(df_treated_filtered,'model0_lca_residuals_G_cl', 'model0_lca_residuals', 'PRESTATIONS CAM', 8, 12, 8, 8, p_001=False, showfliers = False)

In [ ]:
fig, ax = pyspace.plot_getis_by_class(df_treated_filtered,'model0_lca_fitted_values_G_cl', 'model0_lca_fitted_values', 'PRESTATIONS CAM', 8, 12, 8, 8, p_001=False, showfliers = False)

## sensitivity analysis - What happens if we simply aggregate the 5 years?

In [ ]:
df_aggregated = df_treated_filtered.groupby('uuid').sum()

In [ ]:
weights = lps.weights.KNN(cKDTree(get_points_array(df_treated_filtered.geometry)), 32)